Volodymyr Lut, UCU 2019

### Data overview

In this project the data from the [Ukrainian Open Data Portal](https://data.gov.ua) was used.

The dataset contains information about international bus routes and bus stops in Ukraine. Under [this link](https://data.gov.ua/dataset/d5d79cf2-ccc7-40a9-9b7f-4c5cd0d061f9) two files are allocated: `international_routes_stops3.csv` (contains information about stops and schedule of international routes) and `international_routes.csv` (contains information about starting and destination points of the routes, along with schedules). Copies of those datasets (actual on Monday, June 10 2019) are contained under the `raw_data` directory of this repository.

In [12]:
import pandas as pd

In [48]:
stops = pd.read_csv('./raw_data/international_routes_stops3.csv', sep=';', encoding='windows-1251')
stops.head()

,id,rejs_number,stop_city_name,stop_address,stop_code,route_frequency,distance_direct,arrival_direct,departure_direct,time_diff_direct,arrival_return,departure_return,time_diff_return,stop_number,Unnamed: 14
0,CZ1357,1,Київ,"вулиця Набережно-Печерська дорога, 10-а",3200000000,Щоденно,0,20:00,20:10,0,15:30,NaN,0,0.0,NaN
1,CZ1357,1,Київ,"вул. С.Петлюри, 32",3200000000,Щоденно,12,20:30,21:00,0,14:40,15:00,0,1.0,NaN
2,CZ1357,1,Житомир,"вул. Київська, 93",1810100000,Щоденно,140,23:20,23:40,0,12:50,13:00,0,2.0,NaN
3,CZ1357,1,Рівне,"вул. Київська, 40",5610100000,Щоденно,328,2:50,3:10,0,10:30,10:40,0,3.0,NaN
4,CZ1357,1,Львів,"вул. Стрийська, 109",4610100000,Щоденно,539,6:10,6:30,0,8:35,8:50,0,4.0,NaN


In [14]:
routes = pd.read_csv('./raw_data/international_routes.csv')
routes.head()

,permision_delivery_date,n/o,id,march_route_name,company_name,company_name_external,border_crossing,border_crossing_alternative,march_route_time_direct,march_route_time_return,march_route_regularity,license_terms,destionation_country_name,destionation_country_code,march_route_status
0,NaN,3,MD3,"АК АЕРОПОРТ БОРИСПІЛЬ (вул. Бориспільська, 7) ...",ФОП Гергі Є.М.,"S.R.L. ""SARVALTEH AUTO""",Табаки,16:00,16:00,NaN,Щоденно,2020-02-08 00:00:00,Молдова,MD,True
1,NaN,8,MD8,"Балта АС (вул. Уварова, 38а) - Дубосари","ПП Фаворіт 65005, м. Одеса, вул. Бугаївська, 2...","ОАО ""Дубоссарское автотранспортное предприятие...",Платоново,NaN,NaN,NaN,NaN,2020-06-07 00:00:00,Молдова,MD,True
2,05.06.2018,11,RU11,Бахмут-Москва,"ТОВ""Форум Авто""|ФОП""Мелкумян Ю.О.""","ООО""Газавторемонт""",Гоптівка,Плетенівка,16:50,16:50,Щоденно,2019-07-20 00:00:00,Росія,RU,True
3,NaN,12,RU12,"Бахмут (АС вул. Космонавтів,5)-Москва (Новоясе...","ТОВ ""СЕВЕР-АВТО""",без паритету,Плетенівка,NaN,NaN,NaN,NaN,2019-02-28 00:00:00,Росія,RU,True
4,NaN,13,RU13,Бахму - Москва,ФОП Базілєвський В.О.,"ООО ""Горизон-Тур""",Танюшівка,NaN,19:30,15:30,Щоденно,2019-01-07 00:00:00,Росія,RU,True


In this project file `international_routes_stops3.csv` will be used. It should be filtered - in this research I'm interested only in routes, which are heading to Poland. 

**Explanation of useful columns:**

1. *ID* decodes the bus line id - when checking this ID in the `international_routes.csv` dataset, one could determine line operator's comoany name, information about it's license, border crossing point etc.

2. *stop_city_name* together with *stop_address* will be used to determine geolocation of the stop, which will be used for visualisation on maps.

3. *stop_number* is a column which will be used to determine the position of the stop in the route.

In [49]:
### Filtering out Polish routes
pol = stops[stops.id.str[:2].str.upper() == 'PL']
pol = pol.dropna(subset=['stop_city_name', 'stop_address'])
pol.shape

(6446, 15)

In [50]:
import numpy as np

In [51]:
np.unique(pol.id).shape

(482,)

There are **6446 stops** belonging to **482 bus lines** heading to Poland after clearing the data.

### Data Mining

For visualizations we need to know the exact location of every bus stop. In order to scrape this data, [Google Maps API](https://developers.google.com/maps) will be used.

In [52]:
pol = pol.drop("Unnamed: 14", axis=1)
pol = pol.assign(full_address=(pol.stop_city_name + ', ' + pol.stop_address).values)
pol.head()

,id,rejs_number,stop_city_name,stop_address,stop_code,route_frequency,distance_direct,arrival_direct,departure_direct,time_diff_direct,arrival_return,departure_return,time_diff_return,stop_number,full_address
2530,PL944,1,Жовква,пл. Коновальця 1,4622710100,Щоденно,0,4:20,4:40,0,14:55,NaN,0,0.0,"Жовква, пл. Коновальця 1"
2531,PL944,1,Рава Руська,"вул. Двірцева, 4",4622710400,Щоденно,32,5:25,5:30,0,14:05,14:10,0,1.0,"Рава Руська, вул. Двірцева, 4"
2534,PL944,1,Томашів Любельський,"ul. Zamojska, 9",22-600,Щоденно,63,6:05,NaN,-1,11:00,11:30,0,4.0,"Томашів Любельський, ul. Zamojska, 9"
2535,PL944,2,Жовква,пл. Коновальця 1,4622710100,Щоденно,0,21:50,22:10,0,6:25,NaN,0,5.0,"Жовква, пл. Коновальця 1"
2536,PL944,2,Рава Руська,"вул. Двірцева, 4",4622710400,Щоденно,32,22:55,23:00,0,5:35,5:40,0,6.0,"Рава Руська, вул. Двірцева, 4"


In [53]:
np.unique(pol.full_address).shape

(2080,)

In [75]:
import googlemaps
# Uncomment line below and paste your key there
# gmaps = googlemaps.Client(key='<YOUR KEY HERE>')

In [57]:
print(geocode_result)

[{'address_components': [{'long_name': '1', 'short_name': '1', 'types': ['street_number']}, {'long_name': 'Ploshcha Konovalʹtsya', 'short_name': 'Ploshcha Konovalʹtsya', 'types': ['route']}, {'long_name': 'Zhovkva', 'short_name': 'Zhovkva', 'types': ['locality', 'political']}, {'long_name': "Zhovkivs'kyi district", 'short_name': "Zhovkivs'kyi district", 'types': ['administrative_area_level_2', 'political']}, {'long_name': "L'vivs'ka oblast", 'short_name': "L'vivs'ka oblast", 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Ukraine', 'short_name': 'UA', 'types': ['country', 'political']}, {'long_name': '80300', 'short_name': '80300', 'types': ['postal_code']}], 'formatted_address': "Ploshcha Konovalʹtsya, 1, Zhovkva, L'vivs'ka oblast, Ukraine, 80300", 'geometry': {'location': {'lat': 50.0566425, 'lng': 23.9724776}, 'location_type': 'RANGE_INTERPOLATED', 'viewport': {'northeast': {'lat': 50.0579914802915, 'lng': 23.9738265802915}, 'southwest': {'lat': 50.05529351970

In [58]:
from tqdm import tqdm

In [73]:
geocoded_addresses = []
for address in tqdm(np.unique(pol.full_address)):
    geocode_result = gmaps.geocode(address)
    geocoded_addresses.append(geocode_result)
np.savetxt("geocoded.tmp", geocoded_addresses, fmt='%s')

100%|██████████| 2080/2080 [18:28<00:00,  1.77it/s]


array(['Жовква, пл. Коновальця 1', 'Рава Руська, вул. Двірцева, 4',
       'Томашів Любельський, ul. Zamojska, 9'], dtype=object)